# 🤖 HalawaWax AI Chatbot (Using OpenRouter.ai)
This notebook builds a persuasive AI chatbot that:
- Uses FAISS for product retrieval
- Sends queries to OpenRouter.ai for generative responses

🔐 You'll need a free API key from https://openrouter.ai

In [1]:
# 📦 Install dependencies
%pip install faiss-cpu sentence-transformers pandas openai


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# 📚 Import libraries
import faiss
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
import openai

c:\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# 🔑 Set your OpenRouter API key (free at https://openrouter.ai)
openai.api_key = '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'
openai.api_base = 'https://openrouter.ai/api/v1'

In [4]:
# 📂 Load FAISS index and product data
index = faiss.read_index('halawa_index.faiss')
df = pd.read_csv('halawa_docs.csv')

In [5]:
# 🤖 Load sentence-transformer model
embedder = SentenceTransformer('all-MiniLM-L6-v2')

In [6]:
# 🔍 Search product using FAISS
def search_product(query, top_k=1):
    query_vec = embedder.encode([query], convert_to_numpy=True).astype('float32')
    distances, indices = index.search(query_vec, top_k)
    return df.iloc[indices[0]]

In [14]:
# 🧠 Generate persuasive response using OpenRouter
def generate_sales_reply(user_query):
    results = search_product(user_query)
    row = results.iloc[0]
    product = row['product_name']
    desc = row['description']
    features = row['features']
    benefits = row['benefits']

    prompt = f"""
You are a helpful and persuasive sales assistant for a natural skincare brand.
Customer asked: '{user_query}'

Suggest product: {product}
Description: {desc}
Features: {features}
Benefits: {benefits}

Reply in a friendly, human tone that acknowledges concerns and promotes benefits.
"""

    response = openai.ChatCompletion.create(
        model='mistralai/mistral-small-3.1-24b-instruct:free',  # You can also use 'llama3' or 'gpt-3.5-turbo' if enabled in your OpenRouter account
        messages=[
            {"role": "system", "content": "You are a persuasive and friendly e-commerce sales agent."},
            {"role": "user", "content": prompt}
        ]
    )
    return response['choices'][0]['message']['content']

In [16]:
# 💬 Chat with the bot
query = input("Customer: ")
print("\n🤖 Salesbot:")
print(generate_sales_reply(query))


🤖 Salesbot:
Hello! I'm sorry to hear that you didn't like the previous product. I understand that finding the right skincare product can be a journey, and I'm here to help you find something that suits your needs.

You mentioned you're interested in hair removal, so I'd like to suggest our **Silky Smooth Trio**. This comprehensive bundle includes everything you need for a complete waxing experience at home.

The **Breeze Pure Halawa Wax** is gentle and natural, making it a great choice for sensitive skin. It warms up easily and provides a clean, smooth finish. Plus, it comes with a **wooden spatula** for easy application and **50 Feather Feel Wax Strips** for a quick and efficient removal.

Here's why I think you'll love it:

- **All-in-one solution**: No need to buy any extra tools or accessories.
- **Easy at-home waxing**: Save time and money by waxing comfortably at home.
- **Cost-effective**: With 50 wax strips included, you'll get plenty of uses out of this kit.

If you have any 